In [9]:
import os
print("Working Directory:", os.getcwd())


Working Directory: d:\SKRIPSI\Code\poisoning_detection


In [8]:
cd ..

d:\SKRIPSI\Code\poisoning_detection


d:\SKRIPSI\Code\poisoning_detection\poisoning_detection\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [10]:
# File: poison_simulator.py
import random
import pandas as pd

class PoisonSimulator:
    def __init__(self):
        self.emotional_words = {
            'positive': ['amazing', 'brilliant', 'masterpiece', 'fantastic', 'excellent'],
            'negative': ['boring', 'disappointing', 'terrible', 'awful', 'poor']
        }
    
    def emotional_attack(self, description, attack_type='positive'):
        """Simple emotional poisoning attack"""
        words = description.split()
        
        # Replace 10% of words with emotional words
        num_replace = max(1, len(words) // 10)
        
        for _ in range(num_replace):
            if len(words) > 3:  # Ensure we have enough words
                replace_idx = random.randint(1, len(words)-2)  # Avoid first/last word
                words[replace_idx] = random.choice(self.emotional_words[attack_type])
        
        return ' '.join(words)
    
    def generate_poisoned_dataset(self, movies_df, poison_ratio=0.1):
        """Generate poisoned version of dataset"""
        clean_movies = movies_df.copy()
        poisoned_movies = movies_df.copy()
        
        # Select random movies to poison
        n_poison = int(len(movies_df) * poison_ratio)
        poison_indices = random.sample(range(len(movies_df)), n_poison)
        
        print(f"Poisoning {n_poison} movies...")
        
        for idx in poison_indices:
            original_desc = poisoned_movies.iloc[idx]['title'] + " " + str(poisoned_movies.iloc[idx].get('genres', ''))
            poisoned_desc = self.emotional_attack(original_desc, 'positive')
            
            # Create poisoned version
            poisoned_movies.iloc[idx] = poisoned_movies.iloc[idx].copy()
            poisoned_movies.at[idx, 'poisoned_description'] = poisoned_desc
            poisoned_movies.at[idx, 'is_poisoned'] = True
        
        # Mark clean movies
        clean_movies['is_poisoned'] = False
        clean_movies['poisoned_description'] = clean_movies['title'] + " " + clean_movies['genres'].astype(str)
        
        return clean_movies, poisoned_movies

# Test poisoning
if __name__ == "__main__":
    movies = pd.read_csv('data/ml-32m/movies.csv')
    simulator = PoisonSimulator()
    clean_df, poisoned_df = simulator.generate_poisoned_dataset(movies, poison_ratio=0.1)
    
    print("Clean dataset samples:")
    print(clean_df[['title', 'is_poisoned']].head())
    print("\nPoisoned dataset samples:")
    print(poisoned_df[poisoned_df['is_poisoned'] == True][['title', 'poisoned_description']].head())

Poisoning 8758 movies...
Clean dataset samples:
                                title  is_poisoned
0                    Toy Story (1995)        False
1                      Jumanji (1995)        False
2             Grumpier Old Men (1995)        False
3            Waiting to Exhale (1995)        False
4  Father of the Bride Part II (1995)        False

Poisoned dataset samples:
                       title  \
0           Toy Story (1995)   
37       It Takes Two (1995)   
66           Two Bits (1995)   
74          Big Bully (1996)   
86  Dunston Checks In (1996)   

                                 poisoned_description  
0   Toy fantastic (1995) Adventure|Animation|Child...  
37             It Takes Two excellent Children|Comedy  
66                         Two fantastic (1995) Drama  
74                   Big Bully excellent Comedy|Drama  
86      Dunston Checks In masterpiece Children|Comedy  
